<a href="https://colab.research.google.com/github/arinardied/AI_lab/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import tensorflow as tf
import cv2
import tensorflow_datasets as tfds

In [18]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [19]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [20]:
BUFFER_SIZE = 1000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [21]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))
print(validation_inputs.shape)
print(validation_targets.shape)

(6000, 28, 28, 1)
(6000,)


In [22]:
input_size = 784
output_size = 10
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
NUM_EPOCHS = 30
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data,
          epochs=NUM_EPOCHS,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose=2)

Epoch 1/30
540/540 - 8s - 14ms/step - accuracy: 0.8868 - loss: 0.4106 - val_accuracy: 0.9410 - val_loss: 0.2102
Epoch 2/30
540/540 - 5s - 9ms/step - accuracy: 0.9461 - loss: 0.1874 - val_accuracy: 0.9517 - val_loss: 0.1635
Epoch 3/30
540/540 - 4s - 7ms/step - accuracy: 0.9590 - loss: 0.1386 - val_accuracy: 0.9575 - val_loss: 0.1364
Epoch 4/30
540/540 - 5s - 9ms/step - accuracy: 0.9664 - loss: 0.1124 - val_accuracy: 0.9618 - val_loss: 0.1276
Epoch 5/30
540/540 - 5s - 9ms/step - accuracy: 0.9720 - loss: 0.0955 - val_accuracy: 0.9623 - val_loss: 0.1208
Epoch 6/30
540/540 - 4s - 7ms/step - accuracy: 0.9750 - loss: 0.0818 - val_accuracy: 0.9635 - val_loss: 0.1151
Epoch 7/30
540/540 - 4s - 7ms/step - accuracy: 0.9785 - loss: 0.0727 - val_accuracy: 0.9677 - val_loss: 0.1048
Epoch 8/30
540/540 - 5s - 9ms/step - accuracy: 0.9810 - loss: 0.0640 - val_accuracy: 0.9668 - val_loss: 0.1110
Epoch 9/30
540/540 - 4s - 7ms/step - accuracy: 0.9833 - loss: 0.0568 - val_accuracy: 0.9680 - val_loss: 0.1093


In [12]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9771 - loss: 0.0777
Test loss: 0.08. Test accuracy: 97.71%


In [16]:
img = cv2.imread('/content/cyfra.png', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28, 28))
img = cv2.bitwise_not(img)
img = img / 255.0
img = img.reshape(1, 28, 28, 1)
prediction = model.predict(img)
predicted_label = np.argmax(prediction)
print(f'Predykcja modelu: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predykcja modelu: 7
